<a href="https://colab.research.google.com/github/johannes-kk/am207/blob/master/exercises/18_automatic_differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAY 18: Automatic Differentiation 

### AM207: Advanced Scientific Computing

#### Instructor: Weiwei Pan

#### Due: November 3rd, 11:59pm EST

**Names of Group Members**: 
- Matthieu Meeus (matthieu_meeus@g.harvard.edu)
- Nari Johnson njohnson@college.harvard.edu
- Will Seaton (wseaton@g.harvard.edu)
- Maggie Wang (maggiewang@college.harvard.edu)
- Johannes Kolberg (johanneskolberg@g.harvard.edu)
- Alex Spiride (aspiride@college.harvard.edu) 

## Learning Goals:

1. Gain intuition for how to perform reverse and forward mode auto-differentiation.
2. Implement forward-mode auto-differentiation.


### Load necessary libraries

In [ ]:
### Import basic libraries
import numpy as np
import math
import pdb
import matplotlib.pyplot as plt
%matplotlib inline

## Problem 1: Forward versus Backwards Differentiation

In this problem, you'll run through forwards and backwards differentiation for a simple computational graph by hand. These paper-and-pencil exercises are extremely important to do before you sit down to implement anything. We'll work with the same computational graph as in the lecture: $y = f(x_1, x_2) = \ln(x_1) + x_1x_2 - \sin(x_2)$

<img src="https://i.ibb.co/tPKRYf7/computation-graph.jpg">

**Exercise 1:** Compute $\nabla_{\mathrm{x}}f\vert_{x_1=1, x_2=2}$ using forward-mode differentiation.

**Answer:**

Let's write out the evaluation trace and compute the derivative with respect to $x_1$:

\begin{equation*} \begin{aligned}[c] v_{-1} & = x_1 &= 1 \\ v_{0} &= x_2 &= 2 \\ v_1 &= \ln v_{-1} &= \ln 1 \\ v_2 &= v_{-1} \times v_0 &= 1 \times 2 \\ v_3 &= \sin v_0 &= \sin 2 \\ v_4 &= v_1 + v_2 &= 0 + 2 \\ v_5 &= v_4 - v_3 &= 2 - 0.909 \\ y &= v_5 &= 1.091 \end{aligned} \qquad\Longleftrightarrow\qquad \begin{aligned}[c] 
 \dot{v}_{-1} & = \dot{x}_1 &= 1 \\ \dot{v}_{0} &= \dot{x}_2 &= 0 \\ \dot{v}_1 &= \frac{\dot{v}_{-1}}{v_{-1}}  &= 1 \\ \dot{v}_{2} = \dot{v}_{-1}*v_0 + \dot{v}_0*v_{-1}  &= 2 \\ \dot{v}_{3} &= \dot{v}_{0}*\cos v_0 &= 0 \\ \dot{v}_{4} &= \dot{v}_1 + \dot{v}_2 &= 3 \\  \dot{v}_5 &= \dot{v}_4 - \dot{v}_3 &= 3 \\ \dot{y} &= \dot{v}_5 &= 3
 \end{aligned} \end{equation*}


For the derivative with respect to $x_2$, we get:

\begin{equation*} \begin{aligned}[c] v_{-1} & = x_1 &= 1 \\ v_{0} &= x_2 &= 2 \\ v_1 &= \ln v_{-1} &= \ln 1 \\ v_2 &= v_{-1} \times v_0 &= 1 \times 2 \\ v_3 &= \sin v_0 &= \sin 2 \\ v_4 &= v_1 + v_2 &= 0 + 2 \\ v_5 &= v_4 - v_3 &= 2 - 0.909 \\ y &= v_5 &= 1.091 \end{aligned} \qquad\Longleftrightarrow\qquad \begin{aligned}[c]  \dot{v}_{-1} & = \dot{x}_1 &= 0 \\ \dot{v}_{0} &= \dot{x}_2 &= 1 \\ \dot{v}_1 &= \frac{\dot{v}_{-1}}{v_{-1}}  &= 0 \\ \dot{v}_{2} = \dot{v}_{-1}*v_0 + \dot{v}_0*v_{-1}  &= 0 + 1*1 = 1  \\ \dot{v}_{3} &= \dot{v}_{0}*\cos v_0 &= cos(2) \\ \dot{v}_{4} &= \dot{v}_1 + \dot{v}_2 &= 1  \\ v_5 &= \dot{v}_4 - \dot{v}_3 &= 1  - cos(2) \\ y &= v_5 &= 1 -cos(2) = 1.416 \end{aligned} \end{equation*}




**Exercise 2:** Compute $\nabla_{\mathrm{x}}f\vert_{x_1=1, x_2=2}$ using reverse-mode differentiation.

**Answer:**

\begin{equation*}
\begin{aligned}[c]
v_{-1} & = x_1 &= 1 \\
v_{0} &= x_2 &= 2 \\
v_1 &= \ln v_{-1} &= \ln 1 \\
v_2 &= v_{-1} \times v_0 &= 1 \times 2 \\
v_3 &= \sin v_0 &= \sin 2 \\
v_4 &= v_1 + v_2 &= 0 + 2 \\
v_5 &= v_4 - v_3 &= 2 - 0.909 \\
y &= v_5 &= 1.091
\end{aligned}
\qquad\Longleftrightarrow\qquad
\begin{aligned}[c]
\bf{\bar{x}_1} &= \bf{\bar{v}_{-1}} &= \bf{3}\\
\bf{\bar{x}_2} &= \bf{\bar{v}_{0}} &= \bf{1.416}\\
\bar{v}_{-1} &= \bar{v}_{-1} + \bar{v}_1 \frac{\partial v_1}{\partial v_{0}} &= \bar{v}_{-1} + \bar{v}_1 / v_{-1} &= 3 \\
\bar{v}_{0} &= \bar{v}_0 + \bar{v}_2 \frac{\partial v_2}{\partial v_{0}} &= \bar{v}_0 + \bar{v}_2 \times v_{-1} &= 1.416 \\
\bar{v}_{-1} &= \bar{v}_2 \frac{\partial v_2}{\partial v_{-1}} &= \bar{v}_2 \times v_0 &= 2 \\
\bar{v_0} &= \bar{v_3} \frac{\partial v_3}{\partial v_0} &= \bar{v}_3 \times \cos v_0 &= 0.416 \\
\bar{v_2} &= \bar{v_4} \frac{\partial v_4}{\partial v_2} &= \bar{v}_4 \times 1 &= 1 \\
\bar{v_1} &= \bar{v_4} \frac{\partial v_4}{\partial v_1} &= \bar{v}_4 \times 1 &= 1 \\
\bar{v_3} &= \bar{v_5} \frac{\partial v_5}{\partial v_3} &= \bar{v}_5 \times (-1) &= -1 \\
\bar{v_4} &= \bar{v_5} \frac{\partial v_5}{\partial v_4} &= \bar{v}_5 \times 1 &= 1 \\
\bar{v_5} &= \bar{y} &= 1
\end{aligned}
\end{equation*}
 

**Exercise 3:** What are the pros and cons of each mode of differentiation? Describe a concrete set of circumstance where it would be more appropriate to use forwar-mode differentiation. Do the same for reverse mode.

**Answer:**

In forward mode, you only require a single forward pass to calculate both intermediate values and partial derivatives. In reverse mode, you require two passes to first calculate intermediate values then reverse to calculate partial derivatives. However, reverse mode calculations are completely local, only requiring three per-step pieces of information: i) current value of $𝑣_𝑘$ ii) derivative of f with respect to every child $𝑣_𝑘$ iii) derivative of function $ℎ_𝑣$ describing how v depends on 𝑣𝑘, which can be hard-coded. As a result, the trade-offs involve memory storage vs computation time. Forward mode is forward, but requires more memory. Reverse requires less memory, but takes longer due to its extra pass. It would be more appropriate to use forward-mode differentiation for an equation with fewer operations where speed matters. It would be better to use reverse-mode differentiation for more complex functions. 

Additionally, both methods have different computational complexity dependend on the number of input and output. For the forward pass, the number of sweeps required is proportional to the number of input variables, while for the backward pass this is proportional to the number of output variables. As such, it will be computationally better to use the backward pass if you have for instance only one output and multiple variables as input. One example is in a neural network, where you have a one-dimensional loss and you wish to compute the gradient of that loss with respect to a lot of weights. in this case, reverse mode will be the clear winner. 

**Exercise 4:** Suppose we've implemented both forward and reverse mode differentiation. How do we compute second derivatives? For example, can we just run reverse (or forward mode) differentiation twice?

**Answer:**

This definitely seems possible. If you can keep track of the operations needed to compute the first derivatives, you could just apply either the reverse or forward mode once again. 

-> this turns out to be wrong. You first need to compute the reverse and then the forward in order to compute second order derivatives. 

However, calculating the Hessian (second-order derivatives) is more complicated due to interactions between the variables. Our autodiff implementation can’t be applied directly as-is since the interactions add a third dimension to the matrix of calculations.


## Problem 2: Implementing Forward Mode AutoDiff

**Exercise 5:** Implement forward mode auto-differentiation for functions composed of 3 binary arithmetic operations (+, -, *) and four elementary functions (constant, power n, sin, ln).

Verify that your implementation computes derivatives correctly by comparing the derivative your implementation of auto-diff computes versus the derivatives you compute by hand (or have Wolfram Alpha compute).




In [ ]:
'''small example of reverse mode autodiff as implemented in https://github.com/Rufflewind/revad/'''
class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None

    def grad(self):
        if self.grad_value is None:
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value

    #overloading the '+' operator
    def __add__(self, other):
        z = Var(self.value + other.value)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z
    
    #overloading the '*' operator
    def __mul__(self, other):
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z

def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

In [ ]:
'''small example of reverse mode autodiff as implemented in https://github.com/Rufflewind/revad/'''
class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None

    def grad(self):
        if self.grad_value is None:
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value

    #overloading the '+' operator
    def __add__(self, other):
        z = Var(self.value + other.value)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z
    
    #overloading the '*' operator
    def __mul__(self, other):
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z

    #overloading the '-' operator
    def __sub__(self, other):
        z = Var(self.value - other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z

#define elementary function sin
def sin(x):
    z = Var(math.sin(x.value))
    x.children.append((math.cos(x.value), z))
    return z

#define elementary function constant
def const(x):
    z = Var(x.value)
    x.children.append((0, z))
    return z

#define elementary function power n
def pow(x, n):
    z = Var(x.value**n)
    x.children.append((n*x.value, z))
    return z

#define elementary function ln
def log(x):
    z = Var(math.log(x.value))
    x.children.append((1 / x.value, z))
    return z
    

value of x-y + pow(x,3) evaluated at x=0.5, y=4.2: -3.575
forward pass of our implementation: -3.575


In [ ]:
# New example
x = Var(0.5)
y = Var(4.2)
z = const(x-y) + pow(x, 3)
z.grad_value = 1.0

print('value of x-y + pow(x,3) evaluated at x=0.5, y=4.2: {}\nforward pass of our implementation: {}'.format(0.5 - 4.2 + (0.5)**3, z.value))

value of x-y + pow(x,3) evaluated at x=0.5, y=4.2: -3.575
forward pass of our implementation: -3.575


In [ ]:
# Another example
x = Var(0.5)
y = Var(4.2)
z = log(y - x) + sin(pow(x, 3) + y)
z.grad_value = 1.0

print('value of log(y-x) + sin(pow(x,3)+y) evaluated at x=0.5, y=4.2: {}\nforward pass of our implementation: {}'.format(0.382434, z.value))
y.grad()

value of log(y-x) + sin(pow(x,3)+y) evaluated at x=0.5, y=4.2: 0.382434
forward pass of our implementation: 0.38243423425794165


-0.2426370310823537

In [ ]:
£x = Var(0.5)
y = Var(4.2)
z = x * y + sin(x)
z.grad_value = 1.0

print('value of x*y + sin(x) evaluated at x=0.5, y=4.2: {}\nforward pass of our implementation: {}'.format(0.5 * 4.2 + np.sin(0.5), z.value))

value of x*y + sin(x) evaluated at x=0.5, y=4.2: 2.579425538604203
forward pass of our implementation: 2.579425538604203


In [ ]:
x.grad()

1.5

In [ ]:
print('value of dz/dx = y + cos(x) evaluated at x=0.5, y=4.2: {}\nreverse pass of our implementation: {}'.format(4.2 + np.cos(0.5), x.grad_value))

value of dz/dx = y + cos(x) evaluated at x=0.5, y=4.2: 5.077582561890373
reverse pass of our implementation: 1.5


**Exercise 6:** Using your implementations of forward and reverse mode AutoDiff, implement automatic second order derivatives.